In [1]:
!export CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))
!export LD_LIBRARY_PATH=${CUDNN_PATH}/lib

In [2]:
MODEL = "out/bestAlexNet_rgb.keras"
FILE = "_data/test/Mango (P0)__diseased/0012_0005.JPG"
OUTPUT = "vis/"
LAB = False

In [3]:
import os
import click
import keras
from tensorflow.keras import backend as K
from keras import Function as k_func
import cv2
from custom_utils import transform
import tensorflow as tf
from matplotlib import pyplot
import math

2024-07-04 20:36:00.187650: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 20:36:00.202254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 20:36:00.217953: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 20:36:00.217986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 20:36:00.229976: I tensorflow/core/platform/cpu_feature_gua

In [4]:
if not os.path.exists(FILE):
    raise FileNotFoundError(FILE)
    
model_name = os.path.basename(MODEL)
model = keras.models.load_model(MODEL)

file_name = os.path.basename(FILE)
folder = os.path.join(OUTPUT, model_name[:model_name.index('.')], file_name[:file_name.index('.')])

# read input
img = cv2.imread(FILE)
img = transform(img, lab=LAB, rescale=True, smart_resize=True)

# Build the model by calling it on an example input
input_shape = (None,) + img.shape

model(keras.Input(img.shape))
model.summary()

# Ensure the image has the right shape for the model
img = img.reshape((1, *img.shape))  # Adding batch dimension

# Create a new model that will return the outputs of all layers
layer_outputs = [layer.output for layer in model.layers]
intermediate_model = tf.keras.models.Model(inputs=model.inputs, outputs=layer_outputs)

# Get the outputs for the input tensor
outputs = intermediate_model.predict(img)

if not isinstance(outputs, list):
        outputs = [outputs]

2024-07-04 20:36:02.991560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46866 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:06:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_94 (Conv2D)              │ (None, 224, 224, 96)   │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_94          │ (None, 224, 224, 96)   │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_115 (Activation)     │ (None, 224, 224, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 112, 112, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_95 (Conv2D)              │ (None, 112, 112, 256)  │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_95          │ (None, 112, 112, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_116 (Activation)     │ (None, 112, 112, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d (ZeroPadding2D)  │ (None, 58, 58, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_96 (Conv2D)              │ (None, 58, 58, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_96          │ (None, 58, 58, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_117 (Activation)     │ (None, 58, 58, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 29, 29, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_1                │ (None, 31, 31, 512)    │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_97 (Conv2D)              │ (None, 31, 31, 1024)   │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_97          │ (None, 31, 31, 1024)   │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_118 (Activation)     │ (None, 31, 31, 1024)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_2                │ (None, 33, 33, 1024)   │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_98 (Conv2D)              │ (None, 33, 33, 1024)   │     9,438,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_98          │ (None, 33, 33, 1024)   │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_119 (Activation)     │ (None, 33, 33, 1024)   │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,667,844,946 (6.21 GB)

 Trainable params: 833,912,390 (3.11 GB)

 Non-trainable params: 20,164 (78.77 KB)

 Optimizer params: 833,912,392 (3.11 GB)

I0000 00:00:1720118175.101799 1388300 service.cc:145] XLA service 0x152754014ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720118175.101846 1388300 service.cc:153]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-07-04 20:36:15.178409: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-04 20:36:15.870477: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


I0000 00:00:1720118176.834380 1388300 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [ ]:
%matplotlib widget
for layer, output in zip(model.layers, outputs):
    if len(output.shape) != 4:
        continue
    name = layer.name
    print(f"Writing output for layer {name} of image {file_name}")
    print(output.shape)
    output = (output[0] * 255).astype("uint8")
    
    plt_amount = layer.output.shape[-1]
    size = math.ceil(math.sqrt(plt_amount))
    print(f"Creating subplot with size {size}x{size}")
    
    fig, axes = pyplot.subplots(size, size, figsize=(20, 12))
    
    pyplot.setp(axes, xticks=[], yticks=[])
    axes = axes.flatten()
    for i, ax in enumerate(axes):
        if i == plt_amount:
            break
        #ax = pyplot.subplot(size, size, i+1)
        ax.imshow(output[:, :, i], cmap='gray')
    pyplot.savefig(os.path.join(OUTPUT, f"{model_name[:model_name.index('.')]}_{file_name[:file_name.index('.')]}_{name}_.jpg"), bbox_inches='tight', dpi=200)
    print("Saved fig")

Writing output for layer conv2d_94 of image 0012_0005.JPG
(1, 224, 224, 96)
Creating subplot with size 10x10
Saved fig
Writing output for layer batch_normalization_94 of image 0012_0005.JPG
(1, 224, 224, 96)
Creating subplot with size 10x10
Saved fig
Writing output for layer activation_115 of image 0012_0005.JPG
(1, 224, 224, 96)
Creating subplot with size 10x10
Saved fig
Writing output for layer max_pooling2d_7 of image 0012_0005.JPG
(1, 112, 112, 96)
Creating subplot with size 10x10
Saved fig
Writing output for layer conv2d_95 of image 0012_0005.JPG
(1, 112, 112, 256)
Creating subplot with size 16x16
Saved fig
Writing output for layer batch_normalization_95 of image 0012_0005.JPG
(1, 112, 112, 256)
Creating subplot with size 16x16
Saved fig
Writing output for layer activation_116 of image 0012_0005.JPG
(1, 112, 112, 256)
Creating subplot with size 16x16
Saved fig
Writing output for layer max_pooling2d_8 of image 0012_0005.JPG
(1, 56, 56, 256)
Creating subplot with size 16x16
Saved fi

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve